<a href="https://colab.research.google.com/github/MirkoThulke/Python_GoogleCollabs_DividendScreener/blob/main/Mirko_Divididende_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
!pip install PyPDF2
import PyPDF2
import pandas as pd
import re
import string
from operator import itemgetter
import time

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

# COMDIRECT postbox nach "Konto/Depotmitteilung" filtern
# Benoetigte Dokumente : "Dividendengutschrift_", "Ertragsgutschrift_", "Zinsgutschrift_"
# zu entfernende Dokumente : "Ankündigung_Rückzahlung_", "Einlösung_"

# Verkaeufe muessen separat behandelt werden.
# Dafuer in postbox nach "Ordermitteilungen" Dokumenten filtern .
# Benoetigte Dokumente : "Wertpapierabrechnung Verkauf"
Input_Folder  = '/content/drive/MyDrive/Dividenden_Script/Input_PDF'
Output_Folder = '/content/drive/MyDrive/Dividenden_Script/Output_Excel/'

timestr = time.strftime("%Y%m%d-%H%M%S")
text_dump     = ''
text          = ''
filename      = ''
list_items = ['Depotnr.:','Depotbestand','Wertpapier-Bezeichnung','WKN/ISIN','per','Emissionsland:', 'STK','Bruttobetrag:', 'Quellensteuer', 'Dividende', 'Ausschüttung', 'Zinsgutschrift']
myList_all = []
myList_processed = []
Konto         = ''

# Find item in list
def search_list(plist, pstring):
  python_indices  = [index for (index, item) in enumerate(plist) if item == pstring]
  return int(python_indices[0])


# Find first occurance of item in list of lists
def search_list_of_lists(plist, pstring):
  for i in range (0, len(plist)): # lines
      for j in range (0, len(plist[0])): # colums
        if plist[i][j]==pstring:
          found=j
          return j

# convert_string2float
def convert_string2float(pstring):
  temp  = pstring.replace(',','.')  # Convert to float
  temp2 = float(temp) # Convert to float
  return temp2



# Extract data from dump file
def search_text(pfilename, ptext):
    myList1 = []
    myList2 = []
    string1 = ''

    #split string into list based on spaces
    myList1 = ptext.split()

    # remove not required items
    #myList1.remove("abc")

    # remove first element from list : '/SP010000/SP0...'
    myList1.pop(0)
    print(myList1)

    # Depotnummer
    index1 = search_list(myList1, 'Depotnr.:')
    myList2.append(myList1[index1])
    myList2.append(myList1[index1+1])

    # 'Inhaber'
    string1 = ''
    index1 = search_list(myList1, 'Thulke')
    for x in range(2, index1+1):
      string1 = string1 + ' ' + myList1[x]
    myList2.append('Inhaber')
    myList2.append(string1)

    # Datum
    myList2.append(myList1[0])

    # Pruefe ob es sich um Anleihe handelt
    x = ptext.rfind('Zinsgutschrift')
    if x > 0:
        Staatsanleihe_b = True
    else:
        Staatsanleihe_b = False

    x = ptext.rfind('Dividende')
    if x > 0:
        Dividende_b = True
    else:
        Dividende_b = False

    x = ptext.rfind('Devisenkurs:')
    if x > 0:
        Devisenkurs_b = True
    else:
        Devisenkurs_b = False

    x = ptext.rfind('Ausschüttung')
    if x > 0:
        Ausschüttung_b = True
    else:
        Ausschüttung_b = False

    x = ptext.rfind('Ertragsgutschrift')
    if x > 0:
        Ertragsgutschrift_b = True
    else:
        Ertragsgutschrift_b = False

    x = ptext.rfind('Quellensteuer')
    if x > 0:
        Quellensteuer_b = True
    else:
        Quellensteuer_b = False

    # Aktie oder Fond
    if Dividende_b == True or Ausschüttung_b == True or Ertragsgutschrift_b == True  :

        # 'Wertpapier-Bezeichnung'
        string1 = ''
        index1 = search_list(myList1, 'Wertpapier-Bezeichnung')
        index2 = search_list(myList1, 'STK')
        myList2.append(myList1[index1])
        for x in range(index1+5, index2):
          string1 = string1 + ' ' + myList1[x]
        myList2.append(string1)

        # 'WKN/ISIN'
        index1 = search_list(myList1, 'WKN/ISIN')
        index2 = search_list(myList1, 'STK')
        myList2.append(myList1[index1])
        myList2.append(myList1[index2+2])
        myList2.append(myList1[index2])
        myList2.append(myList1[index2+1])

        # 'Emissionsland:'
        string1 = ''
        index1 = search_list(myList1, 'Emissionsland:')
        index2 = search_list(myList1, 'pro')
        myList2.append(myList1[index1])
        for x in range(index1+1, index2-3):
          string1 = string1 + ' ' + myList1[x]
        myList2.append(string1)

        # 'Devisenkurs:'
        if Devisenkurs_b == True:
          index1 = search_list(myList1, 'Devisenkurs:')
          myList2.append(myList1[index1])
          myList2.append(myList1[index1+1])
          myList2.append(convert_string2float(myList1[index1+2])) # convert string to float
        else:
          myList2.append('n.a.')
          myList2.append('n.a.')
          myList2.append('n.a.')

        # 'Dividende' / 'Ausschüttung' / 'Ertragsgutschrift'
        if Dividende_b == True :

          index1 = search_list(myList1, 'Dividende')
          myList2.append(myList1[index1])
          index1 = search_list(myList1, 'Bruttobetrag:')
          myList2.append(myList1[index1])
          myList2.append(myList1[index1+1])
          myList2.append(convert_string2float(myList1[index1+2])) # Convert to float

        elif Ausschüttung_b == True :

          index1 = search_list(myList1, 'Ausschüttung')
          myList2.append(myList1[index1])
          index1 = search_list(myList1, 'Bruttobetrag:')
          myList2.append(myList1[index1])
          myList2.append(myList1[index1+1])
          myList2.append(convert_string2float(myList1[index1+2])) # Convert to float

        elif Ertragsgutschrift_b == True :

          index1 = search_list(myList1, 'Ertragsgutschrift')
          myList2.append(myList1[index1])
          index1 = search_list(myList1, 'Bruttobetrag:')
          myList2.append(myList1[index1])
          myList2.append(myList1[index1+1])
          myList2.append(convert_string2float(myList1[index1+2])) # Convert to float

        else:

          myList2.append('Error')
          index1 = search_list(myList1, 'Error')
          myList2.append('Error')
          myList2.append('Error')
          myList2.append('Error')
          print('ERROR : Weder Dividenden noch Ausschuettung gefunden')


        if Quellensteuer_b == True :

          # 'Quellensteuer'
          index1 = search_list(myList1, 'Quellensteuer')
          myList2.append(myList1[index1])
          myList2.append(myList1[index1-1])
          myList2.append(convert_string2float(myList1[index1-2])) # convert string to float
          myList2.append(myList1[index1+1])
          myList2.append(convert_string2float(myList1[index1+2])) # convert string to float

        else:

          # 'Quellensteuer'
          myList2.append('n.a.')
          myList2.append('n.a.')
          myList2.append('n.a.')
          myList2.append('n.a.')
          myList2.append('n.a.')

    # Staatsanleihe ##################
    elif Staatsanleihe_b == True :
        print("Staatsanleihe")

        # 'Wertpapier-Bezeichnung'
        index1 = search_list(myList1, 'Wertpapier-Bezeichnung')
        myList2.append(myList1[index1])
        myList2.append(myList1[index1+6])

        # 'WKN/ISIN'
        index1 = search_list(myList1, 'WKN/ISIN')
        myList2.append(myList1[index1])
        myList2.append(myList1[index1+8])
        myList2.append('n.a.')
        myList2.append('n.a.')

        # 'Emissionsland:'
        index1 = search_list(myList1, 'Emissionsland:')
        myList2.append(myList1[index1])
        myList2.append(' '+myList1[index1+1])

        # 'Devisenkurs:'
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')

        # 'Dividende' / 'Ausschüttung'
        myList2.append('n.a.')
        index1 = search_list(myList1, 'Bruttobetrag:')
        myList2.append(myList1[index1])
        myList2.append(myList1[index1+1])
        myList2.append(convert_string2float(myList1[index1+2])) # Convert to float

        # 'Quellensteuer'
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')
        myList2.append('n.a.')

        print(myList2)

    # Undefiniertes Dokument
    else:
      print("Undefiniertes Dokument")


    #write result into overall list of lists
    myList_all.append(myList2)

#End of function

for filename in os.listdir(Input_Folder):
    if filename.endswith('.pdf'):
       # Open the PDF file
        with open(os.path.join(Input_Folder, filename), 'rb') as pdf_file:
          print('\n'+filename+'\n')

          # Read the PDF file
          pdf_reader = PyPDF2.PdfReader(pdf_file)

          # Print the number of pages
          #print(len(pdf_reader.pages))

          # Get the first page
          page = pdf_reader.pages[0]

          # Extract the text from the first page
          text = page.extract_text()

          # Print the text
          #print(text)

          # extract data and write into global list : myList_all
          search_text(filename, text)

          # Close the PDF file
          pdf_file.close()
else :
    print('PDFs gelesen ...')




# Create a DataFrame
df1 = pd.DataFrame(myList_all)

# processing
index1 = search_list_of_lists(myList_all, 'Emissionsland:')

myList_processed = sorted(myList_all, key=itemgetter(index1+1))


# Create a DataFrame
df2 = pd.DataFrame(myList_processed)

# Excel export
Konto = myList_processed[0][1] # Konto nummer
excel_datei_name = Konto+'_'+time.strftime("%Y%m%d-%H%M%S")+'.xlsx'
excel_datei = Output_Folder+'/'+excel_datei_name

print(excel_datei)
with pd.ExcelWriter(excel_datei) as writer:
    df1.to_excel(writer, sheet_name='Raw_Data', index=False)
    df2.to_excel(writer, sheet_name='Processed_Data', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Dividendengutschrift_2_St._866993_(NIKE_INC._B)_vom_02.01.2024_7DDC2F.pdf

['02.01.2024', 'Herrn', 'Mirko', 'Andreas', 'Thulke', 'Batiment', 'D3', '82', 'Bis,', 'Avenue', 'de', 'Paris', 'F', '78000', 'VERSAILLES', '97788', '040', 'Depotnr.:', '883189300', 'BLZ:', '200', '411', '44', 'Gutschrift', 'fälliger', 'Wertpapier-Erträge', 'Dividendengutschrift', 'Depotbestand', 'Wertpapier-Bezeichnung', 'WKN/ISIN', 'per', '30.11.2023', '866993', 'NIKE', 'Inc.', 'STK', '2,000', 'US6541061031', 'Registered', 'Shares', 'Class', 'B', 'o.N.', 'Emissionsland:', 'VEREINIGTE', 'STAATEN', 'USD', '0,37', 'Dividende', 'pro', 'Stück', 'für', 'Geschäftsjahr', '01.06.23', 'bis', '31.05.24', 'zahlbar', 'ab', '02.01.2024', 'Quartalsdividende', 'Abrechnung', 'Dividendengutschrift', 'Bruttobetrag:', 'USD', '0,74', '30,000', '%', 'Quellensteuer', 'USD', '0,22', '-', 'Ausmachender', 'Be